In [257]:
using JuMP
import LinearAlgebra    
import SCS
using Hypatia

In [258]:
N, d = 2, 2

(2, 2)

In [259]:
function random_state(d)
    x = randn(ComplexF64, (d, d))
    y = x * x'
    return LinearAlgebra.Hermitian(y / LinearAlgebra.tr(y))
end

#ρ = [random_state(d) for i in 1:N]
#ρ = 0.5 * [LinearAlgebra.Hermitian([[1,0] [0,1]])]
#ρ = 0.5 * [[[1,1] [1,1]],[[1,-1] [-1,1]]] # discriminate between plus and minus
#
ρ = [Hermitian([1 0; 0 0]), 0.5 * Hermitian([1 1; 1 1])] # 0 and plus


ρ

2-element Vector{Hermitian}:
 [1 0; 0 0]
 [0.5 0.5; 0.5 0.5]

In [260]:
model = Model(SCS.Optimizer)
set_silent(model)

In [261]:
E = [@variable(model, [1:d, 1:d] in HermitianPSDCone()) for i in 1:N]

2-element Vector{Hermitian{GenericAffExpr{ComplexF64, VariableRef}, Matrix{GenericAffExpr{ComplexF64, VariableRef}}}}:
 [_[1] _[2] + _[4] im; _[2] - _[4] im _[3]]
 [_[5] _[6] + _[8] im; _[6] - _[8] im _[7]]

In [262]:
@constraint(model, sum(E) == LinearAlgebra.I)

[_[1] + _[5] - 1                  _[2] + _[6] + _[4] im + _[8] im
 _[2] + _[6] - _[4] im - _[8] im  _[3] + _[7] - 1] ∈ Zeros()

In [263]:
@objective(
    model,
    Max,
    sum(real(LinearAlgebra.tr(ρ[i] * E[i])) for i in 1:N) / N,
)

0.5 _[1] + 0.25 _[5] + 0.5 _[6] + 0.25 _[7]

In [264]:
optimize!(model)
assert_is_solved_and_feasible(model)
solution_summary(model)

* Solver : SCS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "solved"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 8.53552e-01
  Dual objective value : 8.53552e-01

* Work counters
  Solve time (sec)   : 1.25759e-02


In [265]:
objective_value(model)

0.8535516271731516

In [266]:
0.5 + 0.25 * sum(LinearAlgebra.svdvals(ρ[1] - ρ[2]))

0.8535533905932737

In [267]:
solution = [value.(e) for e in E]

2-element Vector{Matrix{ComplexF64}}:
 [0.8535516273557899 + 0.0im -0.35355163029489334 + 0.0im; -0.35355163029489334 + 0.0im 0.1464483663886332 + 0.0im]
 [0.1464483661466257 + 0.0im 0.3535516305211815 + 0.0im; 0.3535516305211815 - 0.0im 0.8535516267920372 + 0.0im]

## Solving the dual
$$\min\text{tr[K]}\\
\text{s.c. } K \geq q_i p_i,\quad i \in 1,\ldots,N

$$ 

In [268]:
#model_dual = Model(() -> Hypatia.Optimizer(verbose = false))
model_dual = Model(SCS.Optimizer)
set_silent(model_dual)

In [269]:
ρ[2]

2×2 Hermitian{Float64, Matrix{Float64}}:
 0.5  0.5
 0.5  0.5

In [270]:
K = @variable(model_dual, K[1:d, 1:d], PSD)
for i in 1:N
    @constraint(model_dual, K .>= (ρ[i]/N))
end

In [271]:
@objective(
    model_dual,
    Min,
    LinearAlgebra.tr(K)
)

K[1,1] + K[2,2]

In [272]:
optimize!(model_dual)
assert_is_solved_and_feasible(model_dual)
solution_summary(model_dual)

* Solver : SCS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "solved"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 7.50000e-01
  Dual objective value : 7.49999e-01

* Work counters
  Solve time (sec)   : 1.98077e-02


In [273]:
objective_value(model_dual)

0.749999502359536